# Read in the dataset

In [13]:
import gzip
import os
import humanfriendly
import numpy as np
import pandas as pd

from openclean.data.source.socrata import Socrata

import warnings
warnings.filterwarnings('ignore')

dob_historical_permit_issuance_id = 'bty7-2jhb'
dob_cellular_antenna_filings_id = 'iz2q-9x8d'
dob_C_of_O_id = 'bs8b-p36w'
dob_violations_id = '3h2n-5cm9'

# Set the unique id of the dataset you want to use here
db_id = dob_cellular_antenna_filings_id
dataset = Socrata().dataset(db_id)

datafile = f'./{db_id}.tsv.gz'

# Download file if it doesn't exist
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as  f:
        print('Downloading ...\n')
        dataset.write(f)

fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print(f'Using "{dataset.name}" in file {datafile} of size {fsize}')


Using "DOB Cellular Antenna Filings" in file ./iz2q-9x8d.tsv.gz of size 711.77 KB


## Create data stream

In [14]:
from openclean.pipeline import stream

ds_full = stream(datafile)

## Initial Profile

In [3]:
from openclean.profiling.column import DefaultColumnProfiler

profiles = ds_full.select(columns=['Street', 'Owner’s House City', 'Block', 'Lot']).profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Street,2428526,4,20201,0.008318,11.223448
Owner’s House City,2428526,157034,11547,0.005083,5.538437
Block,2428526,498,13625,0.005612,12.545550
Lot,2428526,507,1718,0.000708,6.659702


### Method to generate a histogram

In [15]:
def get_histogram(data_set, column_name):
    print(f'\n{column_name}:')
    value = data_set.distinct(column_name)
    for rank, val in enumerate(value.most_common()):
        v, freq = val
        print(f'{rank+1:<3} {v} {freq:>10,}')

## Clean Borough data

In [16]:
def clean_borough_name(name):
    boroughs = ['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND']
    if is_empty(name):
        return 'N/A'
    elif name == '1':
        return 'MANHATTAN'
    elif name == '2':
        return 'BRONX'
    elif name == '3':
        return 'BROOKLYN'
    elif name == '4':
        return 'QUEENS'
    elif name == '5':
        return 'STATEN ISLAND'
    elif name.upper() in boroughs:
        return name.upper()
    else:
        return 'N/A'

def clean_borough_data(ds, column_name):
    cleaned_data = ds.update(column_name, lambda x: clean_borough_name(x))
    return cleaned_data

In [17]:
get_histogram(ds_full, 'Borough')


Borough:
1   1      2,161
2   3      1,602
3   4      1,265
4   2        692
5   5        269
6            22
7    EGRESS         11
8    USE          4
9    RELATED DUNNAGE AND RELATED ANTENNAS ON ROOF IN CONFORMANCE WITH TPPN # 5/98. NO CHANGE IN USE          4
10   REMOVAL OF (3) EXISTING ANTENNAS          2
11   GPS ANTENNAS          2
12   OR OCCUPANCY.|01/24/2018 00:00:00          1
13   OR OCCUPANCY.|02/11/2017 00:00:00          1
14   INSTALLATION OF (9) PROPOSED ANTENNAS          1
15   OR EGRESS.|06/27/2017 00:00:00          1
16   EGRESS OR OCCUPANCY.|03/08/2017 00:00:00          1
17   AND INSTALL (1) EQUIPMENT CABINET ON THE ROOF. ALL WORK IN CONFORMANCE WITH TPPN#5/98.  NO CHANGE IN USE          1
18   EGRESS OR OCCUPANCY.|04/19/2017 00:00:00          1
19   RELATED DUNNAGE          1
20   OR OCCUPANCY.|02/10/2018 00:00:00          1
21   (12) RRHS (4 PER SECTOR)          1
22   OCCUPANCY.|02/14/2018 00:00:00          1
23   OR EGRESS.|05/31/2017 00:00:00          1
24  

In [18]:
ds_full = clean_borough_data(ds_full, 'Borough')
get_histogram(ds_full, 'Borough')


Borough:
1   MANHATTAN      2,161
2   BROOKLYN      1,602
3   QUEENS      1,265
4   BRONX        692
5   STATEN ISLAND        269
6   N/A        101


## Clean city data

In [4]:
from fuzzywuzzy import fuzz
import json

from openclean.data.refdata import RefStore
from openclean.function.value.null import is_empty

def clean_city_name(name, valid_city_lookup):
    if is_empty(name):
        return 'N/A'
    return valid_city_lookup.get(name.upper(), 'N/A')


def clean_city_data(ds, column_name):
    # Download the license plate state codes dataset.
    refdata = RefStore()
    refdata\
        .load('encyclopaedia_britannica:us_cities', auto_download=True)\
        .df()\

    # Get set of distinct state codes.
    city_ref = refdata.load('encyclopaedia_britannica:us_cities', auto_download=True).distinct('city')
    # Get list of distinct owner house city names
    city_names = ds.distinct(column_name)
    # Init lookup dictionary for fuzzy matching
    city_ref_lookup = {}
    ref_file_name = 'city_ref_lookup.json'
    # 
    if os.path.isfile(ref_file_name):
        with open(ref_file_name) as f:
            city_ref_lookup = json.load(f)
    # compare city name to each valid city and add to lookup table
    # if the similarity is high enough
    for city in city_names:
        if is_empty(city):
            continue
        name = city.upper()
        if city_ref_lookup.get(name):
            continue
        if (name == 'NYC' or 
              name == 'NY' or
              fuzz.ratio('NY', name) > 70 or
              fuzz.ratio('NYC', name) > 70):
            city_ref_lookup[name] = 'NEW YORK'
            continue
        found_match = False
        for valid_city in city_ref:
            percent_match = fuzz.ratio(valid_city.upper(), name)
            if percent_match > 70:
                found_match = True
                city_ref_lookup[name] = valid_city.upper()
                break
        if not found_match:
            city_ref_lookup[name] = 'N/A'

    cleaned_data = ds.update(column_name, lambda x: clean_city_name(x, city_ref_lookup))
    with open(ref_file_name, 'w') as f:
        json.dump(city_ref_lookup, f)
    return cleaned_data

# Second argument is column name in dataset for city data
ds_full = clean_city_data(ds_full, 'Owner’s House City')

## Clean U.S. Street data

In [6]:
from openclean_geo.address.usstreet import StandardizeUSStreetName

def clean_street_name(name):
    # Replace empty data with 'N/A'
    if is_empty(name):
        return 'N/A'
    # Function to help standardize the street names
    street_func = StandardizeUSStreetName(characters='upper', alphanum=True, repeated=False)
    name = ''.join(street_func.apply([name], threads=None))
    # The conditional statements below are used to try and reduce the remaining
    # number of outlier data by fixing some common errors revealed in the histogram.
    if name == 'CLARKE PLACE EAST':
        name = 'EAST CLARKE PLACE'
    elif name == 'EAST BEDFORD PARK BLVD':
        name = 'BEDFORD PARK BLVD EAST'
    elif name == 'WTC':
        name = 'WORLD TRADE CTR'
    elif name == 'TIME SQ':
        name = 'TIMES SQ'
    elif name == 'PITT':
        name = 'PITT ST'
    elif name == 'BOGARDUS':
        name = 'BOGARDUS PLACE'
    elif name == 'NAGLE':
        name = 'NAGLE AVE'
    elif name == 'SHEPHERD':
        name = 'SHEPHERD AVE'

    split_name = name.split()
    if len(split_name) == 0:
        return 'N/A'
    
    if split_name[-1] in ['SSTREET', 'STRET', 'STREET', 'STREE']:
        split_name[-1] = 'ST'
    elif split_name[-1] == 'PL':
        split_name[-1] = 'PLACE'
    elif split_name[-1].isnumeric():
        split_name.append('ST')
    elif split_name[0] == 'ST':
        split_name[0] = 'SAINT'
    elif split_name[-1] == 'E':
        split_name[-1] = 'EAST'
    elif split_name[-1] == 'W':
        split_name[-1] = 'WEST'
    elif split_name[-1] == 'N':
        split_name[-1] = 'NORTH'
    elif split_name[-1] in ['S', 'SOUIH']:
        split_name[-1] = 'SOUTH'
    elif split_name[-1] in ['BLDV', 'BLV', 'BOULEVARD', 'BOOULEVARD']:
        split_name[-1] = 'BLVD'

    name = ' '.join(split_name)

    return name

def clean_street_data(ds, column_name):
    cleaned_data = ds.update(column_name, lambda x: clean_street_name(x))
    return cleaned_data

ds_full = clean_street_data(ds_full, 'Street')

## Clean block and lot data

In [11]:
import requests
import urllib.parse

# Make a request the url to try and find the block and lot for an address.
# Replace missing data with N/A if it fails to find a value
# This requires you to first clean street and number data
def get_block_and_lot(borough, number, street, block, lot):
    # Return if block and lot are already filled in
    if not is_empty(block) and not is_empty(lot):
        return borough, number, street, block, lot
    if not is_empty(number) and not is_empty(street):
        req = f'https://stevemorse.org/vital/nycblocklot.php?borough={borough.title()}&number={number}&street={urllib.parse.quote(street.title())}'
        r = requests.get(req)
        if r.text == "Callback('?', '?');":
            # Try again without applying title() to street value
            req = f'https://stevemorse.org/vital/nycblocklot.php?borough={borough.title()}&number={number}&street={urllib.parse.quote(street)}'
            r = requests.get(req)
        if r.status_code == 200:
            r_str = r.text.removeprefix('Callback(').removesuffix(');').replace("'", '')
            block_lot = r_str.split(',')
            if is_empty(block):
                block = block_lot[0].strip()
            if is_empty(lot):
                lot = block_lot[1].strip()
    # Return 'N/A' if the web app was unable to find the block and lot data
    # for this input
    block = 'N/A' if block == '?' else block
    if is_empty(block):
        block = 'N/A'
    lot = 'N/A' if lot == '?' else lot
    if is_empty(lot):
        lot = 'N/A'
    return borough, number, street, block, lot

# needed_columns is a list of names for the five columns that hold borough, number, street, block, and lot
def clean_block_and_lot(ds, needed_columns):
    cleaned_data = ds.update(needed_columns, lambda bo, n, s, bl, l: get_block_and_lot(bo, n, s, bl, l))
    return cleaned_data

ds_full = clean_block_and_lot(ds_full, ['BOROUGH','Number','Street','Block','Lot'])

## Profile after clean

In [12]:
profiles = ds_full.select(columns=['Street', 'Owner’s House City', 'Block', 'Lot']).profile(default_profiler=DefaultColumnProfiler)
profiles.stats()

,total,empty,distinct,uniqueness,entropy
Street,2428526,0,8576,0.003531,10.576541
Owner’s House City,2428526,0,1014,0.000418,3.702609
Block,2428526,0,13626,0.005611,12.545786
Lot,2428526,0,1719,0.000708,6.661165
